In [1]:
import os
import pandas as pd

src_dir = os.getcwd()
print(f"SRC DIR: {src_dir}")

subdir1 = "assets"
subdir2 = "results"

target_dir = os.path.join(src_dir, subdir1, subdir2)
print(f"TARGET DIR: {target_dir}")

SRC DIR: /home/nvombat/Desktop/zeetee/src
TARGET DIR: /home/nvombat/Desktop/zeetee/src/assets/results


In [2]:
filename1 = "experiment_data_e1.csv"
file_path1 = os.path.join(target_dir, filename1)

print(f"FILE PATH [E1]: {file_path1}")

FILE PATH [E1]: /home/nvombat/Desktop/zeetee/src/assets/results/experiment_data_e1.csv


In [8]:
df1 = pd.read_csv(file_path1)

df1

,instance_id,encoding_type,num_clauses,num_variables,num_literals,solving_time,status
0,0,e1,8268,441,16960,0.000644,SLV
1,1,e1,8259,441,16933,0.000636,SLV
2,2,e1,13635,602,27902,0.000869,SLV
3,3,e1,20298,803,41387,0.001358,SLV
4,4,e1,20301,785,41419,0.000920,SLV
...,...,...,...,...,...,...,...
175,175,e1,15982,462,32660,0.000446,SLV
176,176,e1,109804,2087,222260,0.002640,SLV
177,177,e1,145305,2503,293202,0.404791,SLV
178,178,e1,186104,2896,375008,1.436484,SLV


In [3]:
filename2 = "experiment_data_e2.csv"
file_path2 = os.path.join(target_dir, filename2)

print(f"FILE PATH [E2]: {file_path2}")

FILE PATH [E2]: /home/nvombat/Desktop/zeetee/src/assets/results/experiment_data_e2.csv


In [10]:
df2 = pd.read_csv(file_path2)

df2

,instance_id,encoding_type,num_clauses,num_variables,num_literals,solving_time,status
0,0,e2,58910,1269,176646,0.001517,SLV
1,1,e2,58892,1262,176594,0.002344,SLV
2,2,e2,58924,1273,176688,0.004985,SLV
3,3,e2,58915,1273,176642,0.001460,SLV
4,4,e2,58934,1280,176698,0.004142,SLV
...,...,...,...,...,...,...,...
175,175,e2,486115,5318,1457752,0.031135,SLV
176,176,e2,486107,5331,1457748,0.033882,SLV
177,177,e2,485482,5089,1456130,0.008088,SLV
178,178,e2,485312,5028,1455681,0.007808,SLV
